In [1]:
import numpy as np
import matplotlib
matplotlib.use('QT5Agg')
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from itertools import product

In [2]:
def print_info(ws, x_train, y_train, x_test, y_test):
    print('Веса:', ws)
    print('Квадратичное отклонение от обучающей выборки:\t\t\t\t', Q(ws, x_train, y_train))
    print('Среднее отклонение от обучающей выборки в процентах:\t\t\t', procQ(ws, x_train, y_train))
    print('Квадратичное отклонение от тестовой выборки:\t\t\t\t', Q(ws, x_test, y_test))
    print('Среднее отклонение от тестовой выборки в процентах:\t\t\t', procQ(ws, x_test, y_test))

In [3]:
data = pd.read_csv('prices.txt')

In [4]:
datax = data[['area', 'rooms']].values
datax = np.insert(datax, 2, -1, axis = 1)
datay = data['price'].values

In [5]:
datax1 = data[['area', 'rooms']].values
datax1 = np.insert(datax1, 2, datax1.T[0] / datax1.T[1], axis = 1)
datax1 = np.insert(datax1, 3, -1, axis = 1)
datay1 = data['price'].values

In [6]:
def fit(ws, x):
    return np.inner(ws, x)

In [7]:
def L(ws, x, y):
    return (y - fit(ws, x)) ** 2

In [8]:
def Q(ws, xs, ys):
    q = 0
    for x, y in zip(xs, ys):
        q += L(ws, x, y)
    return (q / len(ys)) ** 0.5

In [9]:
def procQ(ws, xs, ys):
    pr = 0
    for x, y in zip(xs, ys):
        yhat = fit(ws, x)
        pr += abs(yhat - y) / y * 100
    return pr / len(ys)

In [10]:
def gradL(ws, x, y):
    return -1 * (y - fit(ws, x)) * x

In [11]:
def gradQ(ws, xs, ys):
    g = 0
    for x, y in zip(xs, ys):
        g += gradL(ws, x, y)
    return g

In [12]:
def de(xs, ys, gen_size = 20, F = 1.2, p = 0.7, n = 600):
    gen = (np.random.rand(gen_size, xs.shape[1]) - 0.5) * 10000
    for _ in range(n):
        new_gen = gen.copy()
        for i in range(gen_size):
            vs = np.random.randint(0, gen_size, size=3)
            while len(np.unique(vs)) != 3 or i in vs:
                vs = np.random.randint(0, gen_size, size=3)
            
            v = gen[vs[0]] + F * (gen[vs[1]] - gen[vs[2]])
            for j in range(xs.shape[1]):
                if np.random.rand(1) > p:
                    v[j] = gen[i][j]
            if Q(v, xs, ys) < Q(gen[i], xs, ys):
                new_gen[i] = v
        gen = new_gen
        
    qs = np.array(list(map(lambda v: Q(v, xs, ys), gen)))
    return gen[qs.argmin()]

In [13]:
def gd(xs, ys, n = 5000, eps = 1e-8):
    ws = (np.random.rand(xs.shape[1]) - 0.5) * 1000
    
    H = np.zeros((xs.shape[1],xs.shape[1]))
    for i in range(xs.shape[1]):
        for j in range(xs.shape[1]):
            H[i][j] = sum(map(lambda x: x[i] * x[j], xs))
        
    p = 0
    b = 0
    for i in range(n):
        alpha = 0
        if (i > 0):
            b = np.inner(gr, gr)
        gr = gradQ(ws, xs, ys)
        if (i > 0):
            b = np.inner(gr, gr) / b
        p = -gr + b * p
        alpha = -1 * np.inner(gr, p) / np.inner(np.dot(p, H), p)
        
        ws_next = ws + alpha * p        
        if norm(ws_next - ws, 2) < eps:
            return ws_next, i
        ws = ws_next
    return ws, -1

In [14]:
def LOO(xs, ys, method, **kwargs):
    err = 0
    for i in range(len(xs)):
        ws = method(np.append(xs[:i], xs[i + 1:], axis = 0), 
                    np.append(ys[:i], ys[i + 1:], axis = 0),
                    **kwargs)
        err += L(ws, xs[i], ys[i]) / len(xs)
    return err, ws

In [15]:
# def find_ws_with_de(xs, ys, N):
#     gen_size = zip(['gen_size'] * 4, range(20, 100, 50))
#     F = [('F', 0.8 + 0.3 * i) for i in range(4)]
#     p = [('p', 0.2 + 0.3 * i) for i in range(3)]
#     err = []
#     params = []
#     for args in map(dict, product(F, p, gen_size)):
#         print(args)
#         args['n'] = N // args['gen_size']
#         print(LOO(xs, ys, de, **args))
#     return params, err

In [16]:
def find_ws_with_gd(xs, ys):
    return gd(xs, ys)

In [17]:
seed = np.random.randint(100)

In [18]:
test_size = 10
x_train1, x_test1, y_train1, y_test1 = train_test_split(datax1, datay1, 
                                                test_size = test_size, train_size = len(data) - test_size,
                                                random_state = seed)

In [19]:
test_size = 10
x_train, x_test, y_train, y_test = train_test_split(datax, datay, 
                                                test_size = test_size, train_size = len(data) - test_size,
                                                random_state = seed)

In [20]:
def draw_train_test():
    fig = plt.figure()
    plt.plot()
    ax = fig.add_subplot(111, projection ='3d')
    ax.plot(x_train.T[0],x_train.T[1],y_train, 'ro')
    ax.plot(x_test.T[0],x_test.T[1],y_test, 'bo')

    plt.show()

In [21]:
draw_train_test()

In [22]:
ws_gr, i = find_ws_with_gd(x_train, y_train)
print('Метод сопряженных градиентов')
print_info(ws_gr, x_train, y_train, x_test, y_test)
print('Количество итераций:', i)

print('---------------------------------')

ws_gr1, i1 = find_ws_with_gd(x_train1, y_train1)
print('Метод сопряженных градиентов')
print_info(ws_gr1, x_train1, y_train1, x_test1, y_test1)
print('Количество итераций:', i)

Метод сопряженных градиентов
Веса: [   131.82972511  -3378.11050846 -82302.25859416]
Квадратичное отклонение от обучающей выборки:				 64217.5197782
Среднее отклонение от обучающей выборки в процентах:			 15.4673128677
Квадратичное отклонение от тестовой выборки:				 64659.2616053
Среднее отклонение от тестовой выборки в процентах:			 14.9972126544
Количество итераций: 4
---------------------------------
Метод сопряженных градиентов
Веса: [  2.27240852e+02  -8.06918825e+04  -3.14783375e+02  -3.38542883e+05]
Квадратичное отклонение от обучающей выборки:				 62698.4355506
Среднее отклонение от обучающей выборки в процентах:			 14.443983628
Квадратичное отклонение от тестовой выборки:				 65771.2050128
Среднее отклонение от тестовой выборки в процентах:			 15.8952606978
Количество итераций: 4


In [23]:
ws_de = de(x_train, y_train, 20, 1, 0.7, 500)
print('Метод дифференциальной эволюции')
print_info(ws_de, x_train, y_train, x_test, y_test)

print('---------------------------------')

ws_de1 = de(x_train1, y_train1, 20, 1, 0.7, 500)
print('Метод дифференциальной эволюции')
print_info(ws_de1, x_train1, y_train1, x_test1, y_test1)

Метод дифференциальной эволюции
Веса: [   131.82972558  -3378.11067591 -82302.25811559]
Квадратичное отклонение от обучающей выборки:				 64217.5197782
Среднее отклонение от обучающей выборки в процентах:			 15.4673128515
Квадратичное отклонение от тестовой выборки:				 64659.2615249
Среднее отклонение от тестовой выборки в процентах:			 14.9972126198
---------------------------------
Метод дифференциальной эволюции
Веса: [  2.27016505e+02  -8.05065489e+04  -3.14113340e+02  -3.37963949e+05]
Квадратичное отклонение от обучающей выборки:				 62698.4449165
Среднее отклонение от обучающей выборки в процентах:			 14.4442058154
Квадратичное отклонение от тестовой выборки:				 65770.1518849
Среднее отклонение от тестовой выборки в процентах:			 15.8915087072


In [24]:
ws_ex = np.dot(np.dot(np.linalg.inv(np.dot(x_train.T, x_train)), x_train.T), y_train)
print('Точный ответ')
print_info(ws_ex, x_train, y_train, x_test, y_test)
""
print('---------------------------------')

ws_ex1 = np.dot(np.dot(np.linalg.inv(np.dot(x_train1.T, x_train1)), x_train1.T), y_train1)
print('Точный ответ')
# ws_ex1 = [   139.21067402,  -8738.01911233, 0, -89597.9095428 ]
print_info(ws_ex1, x_train1, y_train1, x_test1, y_test1)

Точный ответ
Веса: [   131.82972511  -3378.11050846 -82302.25859416]
Квадратичное отклонение от обучающей выборки:				 64217.5197782
Среднее отклонение от обучающей выборки в процентах:			 15.4673128677
Квадратичное отклонение от тестовой выборки:				 64659.2616053
Среднее отклонение от тестовой выборки в процентах:			 14.9972126544
---------------------------------
Точный ответ
Веса: [  2.27240852e+02  -8.06918825e+04  -3.14783375e+02  -3.38542883e+05]
Квадратичное отклонение от обучающей выборки:				 62698.4355506
Среднее отклонение от обучающей выборки в процентах:			 14.443983628
Квадратичное отклонение от тестовой выборки:				 65771.2050129
Среднее отклонение от тестовой выборки в процентах:			 15.8952606979


In [25]:
def draw_plane(ws):
    point  = np.array([0,0, -ws[2]])
    normal = np.array([ws[0], ws[1], -1])
    
    d = -point.dot(normal)
    
    nn = 6
    xx = np.arange(0, 5000, 5000 / nn)
    yy = np.arange(0, 5, 5 / nn)
    
    xx, yy = np.meshgrid(xx, yy)
    
    z = (-normal[0] * xx - normal[1] * yy - d) * 1. /normal[2]
    
    ax = plt.figure().gca(projection='3d')
    ax.plot_surface(xx, yy, z)
    
    
    for x, y in zip(x_test, y_test):
        ax.plot([x[0], x[0]], [x[1], x[1]], [y, fit(ws, x)], 'k--')
    
    ax.plot(x_train.T[0],x_train.T[1],y_train, 'ro')
    ax.plot(x_test.T[0],x_test.T[1],y_test, 'bo')
    
    plt.show()

In [26]:
draw_plane(ws_gr)